In [40]:
import numpy as np
from sklearn.manifold import TSNE

 
import pandas as pd
country = pd.read_csv('Data/Countrys.csv')
country.head()

,Unnamed: 0,content,sentiment
0,0,Yo espero tranquila. Si gana de nvo alg partid...,NEU
1,1,"Los fracasos de Michelle, Cristina y Dilma mue...",N
2,2,Debemos orar por los hnos venezolanos gente q ...,N
3,3,Me dijo que no le había gustado no se cuantas ...,NEU
4,4,@susanagkp jajaja yo fui el viernes ! precavid...,N


In [41]:
countryContSen = country[['content','sentiment']].copy()
countryContSen['sentiment'] = countryContSen.sentiment.map({'NONE':0,'N':1,'NEU':2, 'P':3})

In [42]:
# Importar el contador de vectorizacion e inicializarlo
from sklearn.feature_extraction.text import CountVectorizer
count_vector = CountVectorizer()

In [43]:
print(countryContSen.groupby('sentiment').size())

sentiment
0     694
1    1410
2     532
3    1039
dtype: int64


In [44]:
countW = []
# Conteo de palabras
for i in countryContSen.content.tolist():
    countW.append(len(i.split()))
dfc = countryContSen.assign(countW = countW)
dfc

,content,sentiment,countW
0,Yo espero tranquila. Si gana de nvo alg partid...,2,23
1,"Los fracasos de Michelle, Cristina y Dilma mue...",1,24
2,Debemos orar por los hnos venezolanos gente q ...,1,25
3,Me dijo que no le había gustado no se cuantas ...,2,24
4,@susanagkp jajaja yo fui el viernes ! precavid...,1,26
...,...,...,...
3670,Quiero como tener carro para hacer todo rápido,2,8
3671,Tristeza más hp me da siempre en año nuevo,1,9
3672,no encuentro ninguna buena serie en NF.,1,7
3673,@Alphasniper97 Alpha suerte y ten un feliz año...,3,9


In [45]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
# prepara los datos 
X = dfc[['countW']].values
y = dfc['sentiment'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [46]:
#Se crea el clasificador
n_neighbors = 16

knn = KNeighborsClassifier(n_neighbors)
knn.fit(X_train, y_train)
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(knn.score(X_test, y_test)))

Accuracy of K-NN classifier on training set: 0.33
Accuracy of K-NN classifier on test set: 0.28


In [47]:
#precision del modelo
pred = knn.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

[[ 37  96   5  48]
 [ 83 163  14  91]
 [ 30  65   3  42]
 [ 57 115  17  53]]
              precision    recall  f1-score   support

           0       0.18      0.20      0.19       186
           1       0.37      0.46      0.41       351
           2       0.08      0.02      0.03       140
           3       0.23      0.22      0.22       242

    accuracy                           0.28       919
   macro avg       0.21      0.23      0.21       919
weighted avg       0.25      0.28      0.26       919



In [48]:
# Clasificación obtenida
k_range = range(1, 20)
scores = []
for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    scores.append(knn.score(X_test, y_test))

In [49]:
import pickle

In [50]:
# pickling the vectorizer
pickle.dump(count_vector, open('Modelos/vectorizerKNN.sav', 'wb'))

# pickling the model
pickle.dump(knn, open('Modelos/classifierKNN.sav', 'wb'))